In [2]:
import numpy as np
from pymatgen.core import Structure
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import json

2023-11-26 23:02:53.659350: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-26 23:02:53.862327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 23:02:54.731751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 23:02:54.731780: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 23:02:54.737378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
with open("/root/public/bandgap_json/DFT_bandgaptrain1.json", "r") as f:
    data_train = json.load(f)

In [4]:
structures = []
gaps = []

for item in data_train:
    structure = Structure.from_dict(item['structure'])
    gap = item['real_gap'] - item['bandgap']
    structures.append(structure)
    gaps.append(gap)

In [5]:
#5是最低截断半径，用以判断哪两个原子之间有键连接

In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 设置参数列表
nfeat_bond_list = [10, 15, 20, 25, 30]
r_cutoff_list = [6, 7, 8, 9, 10]

# 初始化结果记录
results = []

# 循环训练模型
for nfeat_bond in nfeat_bond_list:
    for r_cutoff in r_cutoff_list:
        # 设置参数
        gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
        gaussian_width = 0.5
        graph_converter = CrystalGraph(cutoff=r_cutoff)
        
        # 初始化模型
        model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)
        
        # 训练模型
        model.train(structures, gaps, epochs=100)
        
        # 预测结果
        predictions = []
        for structure in structures:
            prediction = model.predict_structure(structure)
            predictions.append(prediction[0])
        
        # 计算评估指标
        mse = mean_squared_error(gaps, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(gaps, predictions)
        r2 = r2_score(gaps, predictions)
        
        # 记录结果
        results.append({
            'nfeat_bond': nfeat_bond,
            'r_cutoff': r_cutoff,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })



/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 21s 21s/step - loss: 0.1535
Epoch 2/100
1/1 [==============================] - 1s 779ms/step - loss: 0.1532
Epoch 3/100
1/1 [==============================] - 1s 782ms/step - loss: 0.1530
Epoch 4/100
1/1 [==============================] - 1s 769ms/step - loss: 0.1525
Epoch 5/100
1/1 [==============================] - 1s 777ms/step - loss: 0.1524
Epoch 6/100
1/1 [==============================] - 1s 778ms/step - loss: 0.1523
Epoch 7/100
1/1 [==============================] - 1s 775ms/step - loss: 0.1519
Epoch 8/100
1/1 [==============================] - 1s 772ms/step - loss: 0.1515
Epoch 9/100
1/1 [==============================] - 1s 777ms/step - loss: 0.1512
Epoch 10/100
1/1 [==============================] - 1s 781ms/step - loss: 0.1508
Epoch 11/100
1/1 [==============================] - 1s 773ms/step - loss: 0.1502
Epoch 12/100
1/1 [==============================] - 1s 771ms/step - loss: 0.1495
Epoch 13/100
1/1 [====================

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 19s 19s/step - loss: 0.1535
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.1537
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.1528
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.1524
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.1524
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.1520
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.1515
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.1511
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.1508
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.1502
Epoch 11/100
1/1 [==============================] - 1s 1s/step - loss: 0.1493
Epoch 12/100
1/1 [==============================] - 1s 1s/step - loss: 0.1484
Epoch 13/100
1/1 [==============================] - 1s 1s/step - loss: 

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

Epoch 1/100
1/1 [==============================] - 24s 24s/step - loss: 0.1532
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.1531
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.1526
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.1522
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.1520
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.1515
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.1510
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.1505
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.1499
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.1490
Epoch 11/100
1/1 [==============================] - 1s 1s/step - loss: 0.1479
Epoch 12/100
1/1 [==============================] - 1s 1s/step - loss: 0.1467
Epoch 13/100
1/1 [==============================] - 1s 1s/step - loss: 

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 22s 22s/step - loss: 0.1530
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 0.1533
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 0.1525
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 0.1524
Epoch 5/100
1/1 [==============================] - 2s 2s/step - loss: 0.1523
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 0.1517
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 0.1512
Epoch 8/100
1/1 [==============================] - 2s 2s/step - loss: 0.1509
Epoch 9/100
1/1 [==============================] - 2s 2s/step - loss: 0.1505
Epoch 10/100
1/1 [==============================] - 2s 2s/step - loss: 0.1499
Epoch 11/100
1/1 [==============================] - 2s 2s/step - loss: 0.1490
Epoch 12/100
1/1 [==============================] - 2s 2s/step - loss: 0.1482
Epoch 13/100
1/1 [==============================] - 2s 2s/step - loss: 

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 24s 24s/step - loss: 0.1537
Epoch 2/100
1/1 [==============================] - 3s 3s/step - loss: 0.1532
Epoch 3/100
1/1 [==============================] - 3s 3s/step - loss: 0.1531
Epoch 4/100
1/1 [==============================] - 3s 3s/step - loss: 0.1527
Epoch 5/100
1/1 [==============================] - 3s 3s/step - loss: 0.1525
Epoch 6/100
1/1 [==============================] - 3s 3s/step - loss: 0.1524
Epoch 7/100
1/1 [==============================] - 3s 3s/step - loss: 0.1522
Epoch 8/100
1/1 [==============================] - 3s 3s/step - loss: 0.1518
Epoch 9/100
1/1 [==============================] - 3s 3s/step - loss: 0.1514
Epoch 10/100
1/1 [==============================] - 3s 3s/step - loss: 0.1511
Epoch 11/100
1/1 [==============================] - 3s 3s/step - loss: 0.1507
Epoch 12/100
1/1 [==============================] - 3s 3s/step - loss: 0.1501
Epoch 13/100
1/1 [==============================] - 3s 3s/step - loss: 

In [16]:
# 打印结果
for result in results:
    print(f"nfeat_bond: {result['nfeat_bond']}, r_cutoff: {result['r_cutoff']}, MSE: {result['mse']}, RMSE: {result['rmse']}, MAE: {result['mae']}, R2: {result['r2']}")


nfeat_bond: 10, r_cutoff: 6, MSE: 0.019967974872980205, RMSE: 0.1413080849526318, MAE: 0.07920198556232247, R2: 0.8695552210475589
nfeat_bond: 10, r_cutoff: 7, MSE: 0.02133869439120342, RMSE: 0.1460776998422532, MAE: 0.07920198556232247, R2: 0.860600722371663
nfeat_bond: 10, r_cutoff: 8, MSE: 0.011160955786948644, RMSE: 0.10564542482733762, MAE: 0.07920198556232247, R2: 0.9270888299996547
nfeat_bond: 10, r_cutoff: 9, MSE: 0.014913831655681128, RMSE: 0.12212219968409155, MAE: 0.07920198556232247, R2: 0.9025724197854578
nfeat_bond: 10, r_cutoff: 10, MSE: 0.021019840155597387, RMSE: 0.14498220634132103, MAE: 0.07920198556232247, R2: 0.8626837012689352
nfeat_bond: 15, r_cutoff: 6, MSE: 0.016743513218297687, RMSE: 0.12939672800460483, MAE: 0.07920198556232247, R2: 0.8906196600035013
nfeat_bond: 15, r_cutoff: 7, MSE: 0.022134389038193012, RMSE: 0.14877630536544795, MAE: 0.07920198556232247, R2: 0.8554026883696948
nfeat_bond: 15, r_cutoff: 8, MSE: 0.013260232070370543, RMSE: 0.115153081028561

In [9]:
def get_best_r2(results):
    # 找到R2最大的结果
    best_result = max(results, key=lambda result: result['r2'])
    return best_result

def get_best_rmse(results):
    # 计算每个结果的RMSE
    for result in results:
        result['rmse'] = np.sqrt(result['mse'])
    
    # 找到RMSE最小的结果
    best_result = min(results, key=lambda result: result['rmse'])
    return best_result


In [17]:
best_r2_result = get_best_r2(results)
print(f"Best R2: {best_r2_result['r2']}, nfeat_bond: {best_r2_result['nfeat_bond']}, r_cutoff: {best_r2_result['r_cutoff']}")

best_rmse_result = get_best_rmse(results)
print(f"Best RMSE: {best_rmse_result['rmse']}, nfeat_bond: {best_rmse_result['nfeat_bond']}, r_cutoff: {best_rmse_result['r_cutoff']}")


Best R2: 0.9270888299996547, nfeat_bond: 10, r_cutoff: 8
Best RMSE: 0.10564542482733762, nfeat_bond: 10, r_cutoff: 8


In [ ]:
def get_best_mae(results):
    # 计算每个结果的MAE
    for result in results:
        result['mae'] = mean_absolute_error(gaps, predictions)
    
    # 找到MAE最小的结果
    best_result = min(results, key=lambda result: result['mae'])
    return best_result

In [18]:
best_mae_result = get_best_mae(results)
print(f"Best MAE: {best_mae_result['mae']}, nfeat_bond: {best_mae_result['nfeat_bond']}, r_cutoff: {best_mae_result['r_cutoff']}")


Best MAE: 0.07920198556232247, nfeat_bond: 10, r_cutoff: 6


In [19]:
!nvidia-smi

Sun Nov 26 22:41:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:00:08.0 Off |                    0 |
| N/A   31C    P0              22W / 300W |      2MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 设置参数列表
nfeat_bond_list2 = [10]
r_cutoff_list2 = [10,12]#越高，计算时间越长，更多的边，键长信息将被处理

# 初始化结果记录
results2 = []

# 循环训练模型
for nfeat_bond in nfeat_bond_list2:
    for r_cutoff in r_cutoff_list2:
        # 设置参数
        gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
        gaussian_width = 0.5
        graph_converter = CrystalGraph(cutoff=r_cutoff)
        
        # 初始化模型
        model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)
        
        # 训练模型
        model.train(structures, gaps, epochs=100)
        
        # 预测结果
        predictions = []
        for structure in structures:
            prediction = model.predict_structure(structure)
            predictions.append(prediction[0])
        
        # 计算评估指标
        mse = mean_squared_error(gaps, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(gaps, predictions)
        r2 = r2_score(gaps, predictions)
        
        # 记录结果
        results2.append({
            'nfeat_bond': nfeat_bond,
            'r_cutoff': r_cutoff,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })

Epoch 1/100
1/1 [==============================] - 22s 22s/step - loss: 0.1537
Epoch 2/100
1/1 [==============================] - 4s 4s/step - loss: 0.1537
Epoch 3/100
1/1 [==============================] - 4s 4s/step - loss: 0.1532
Epoch 4/100
1/1 [==============================] - 4s 4s/step - loss: 0.1529
Epoch 5/100
1/1 [==============================] - 4s 4s/step - loss: 0.1528
Epoch 6/100
1/1 [==============================] - 4s 4s/step - loss: 0.1527
Epoch 7/100
1/1 [==============================] - 4s 4s/step - loss: 0.1524
Epoch 8/100
1/1 [==============================] - 4s 4s/step - loss: 0.1521
Epoch 9/100
1/1 [==============================] - 4s 4s/step - loss: 0.1519
Epoch 10/100
1/1 [==============================] - 4s 4s/step - loss: 0.1517
Epoch 11/100
1/1 [==============================] - 4s 4s/step - loss: 0.1512
Epoch 12/100
1/1 [==============================] - 4s 4s/step - loss: 0.1507
Epoch 13/100
1/1 [==============================] - 4s 4s/step - loss: 

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 26s 26s/step - loss: 0.1534
Epoch 2/100
1/1 [==============================] - 7s 7s/step - loss: 0.1533
Epoch 3/100
1/1 [==============================] - 7s 7s/step - loss: 0.1531
Epoch 4/100
1/1 [==============================] - 7s 7s/step - loss: 0.1528
Epoch 5/100
1/1 [==============================] - 7s 7s/step - loss: 0.1528
Epoch 6/100
1/1 [==============================] - 7s 7s/step - loss: 0.1527
Epoch 7/100
1/1 [==============================] - 7s 7s/step - loss: 0.1525
Epoch 8/100
1/1 [==============================] - 7s 7s/step - loss: 0.1522
Epoch 9/100
1/1 [==============================] - 7s 7s/step - loss: 0.1521
Epoch 10/100
1/1 [==============================] - 7s 7s/step - loss: 0.1518
Epoch 11/100
1/1 [==============================] - 7s 7s/step - loss: 0.1514
Epoch 12/100
1/1 [==============================] - 7s 7s/step - loss: 0.1509
Epoch 13/100
1/1 [==============================] - 7s 7s/step - loss: 

In [8]:
# 打印结果
for result in results2:
    print(f"nfeat_bond: {result['nfeat_bond']}, r_cutoff: {result['r_cutoff']}, MSE: {result['mse']}, RMSE: {result['rmse']}, MAE: {result['mae']}, R2: {result['r2']}")


nfeat_bond: 10, r_cutoff: 10, MSE: 0.020761438802568193, RMSE: 0.1440883021017605, MAE: 0.1092880264815893, R2: 0.8643717596519871
nfeat_bond: 10, r_cutoff: 12, MSE: 0.0441800792883566, RMSE: 0.210190578495699, MAE: 0.1498537179177882, R2: 0.7113848192653066
